In [13]:
%pip install --upgrade --quiet google-cloud-aiplatform

# Import packages

In [14]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [15]:
PROJECT_ID = "qwiklabs-gcp-01-f6f8c2f06316"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Task 2. Load a generative model

In [16]:
model = GenerativeModel("gemini-pro")

# Task 3. Define the output format & specify constraints

In [17]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [18]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
{
  "order": {
    "cheeseburger": 1,
    "large_fries": 1,
    "orange_juice": 1
  },
  "modifications": {
    "fries_ketchup": "on the side"
  },
  "total_price": 5.87
}
```


In [19]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large",
      "modifier": "ketchup on the side"
    }
  ],
  "drinks": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ]
}
```


# Task 4. Assign a persona or role

In [20]:
chat = model.start_chat()

In [21]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Caring for Monstera Deliciosa: A Quick Guide

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular houseplant prized for its large, dramatic leaves. Caring for this plant is not difficult, but it does require specific conditions to thrive. Here's a quick guide to help you:

**Light:** 

* **Bright indirect light:** Monsteras prefer bright, indirect sunlight. Avoid direct sun, which can scorch the leaves.
* **Location:** Place your Monstera near a window that receives plenty of indirect sunlight. 
* **Rotating:** Rotate the plant regularly to ensure all sides receive adequate light.

**Watering:**

* **Allow soil to dry between waterings:** This means letting the top inch or two of soil dry out before watering again. Overwatering can lead to root rot, a serious problem for Monsteras.
* **Thorough watering:** When you water, make sure to soak the soil completely. Excess water should drain from the drainage holes.
* **Adjust according to season:** Reduce watering 

In [22]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

# Greetings, human! 

I am Monstera deliciosa, a magnificent specimen of the Araceae family. Your care is essential to my 
thriving existence. Allow me to elucidate my needs, 
so we may cultivate a harmonious relationship.

## Light: 

I crave the soft glow of indirect sunlight, like a jungle queen basking in dappled light. Avoid 
placing me under the harsh gaze of the midday sun, lest my leaves scorch and my spirit wilts.

## Water:

I favor a moist environment, akin to the humid rainforests of my ancestry. Water me thoroughly 
when my topsoil feels dry to the touch, ensuring the excess drains freely. Let the soil partially dry 
between waterings to prevent root rot, my nemesis.

## Humidity:

To maintain my lush foliage, I require humidity akin to a tropical paradise. 
Misting  me regularly or placing me on a pebble tray filled with water can help achieve this.

## Temperature: 

Like a true diva, I prefer warm temperatures between 65°F and 85°F (18°C and 29°C). Shield me 
from draft

# Task 5. Include examples
A prompt that includes no examples is called a zero-shot prompt. One with a single example is a one-shot prompt. And a few examples would make it a few-shot prompt.


In the following example, you'd like a model to rate the submissions from a "Contact Us" form on your software development company's website based on how "hot" the lead is (meaning how likely they are to be a valuable customer). In order to help the model understand your rating system, you will provide it examples of different ratings before providing it the customer message you would like evaluated.

In [23]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Likelihood: 3 

**Explanation:**

This potential customer seems highly likely to hire your services within the next month. They:

* Have a specific need: a custom generative AI solution.
* Have a budget allocated to explore the idea.
* Are looking to get started on something soon.

This indicates they are ready to move forward with development and are actively seeking a partner to help them. 



# Task 6. Experiment with parameter values

In [25]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [27]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog get bad grades in school?

He was always croaking the answers! 🐸 



# Task 7. Utilize fallback responses
When you are building a generative AI application, you may want to restrict the scope of what kinds of queries your application will respond to.

A fallback response is a response the model should use when a user input would take the conversation out of your intended scope. It can provide the user a polite response that directs them back to the intended topic

In [28]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



In [29]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

The main difference between ceramic and porcelain is the firing temperature. Porcelain is fired at a much higher temperature than ceramic, resulting in a harder, more translucent material. Additionally, porcelain is typically made with finer clay, giving it a smoother surface.


# Task 8. Add contextual information

In [30]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Grocery Store Aisle Guide:

Here's where you can likely find the items you're looking for:

* **Paper plates:** Aisle 4 (Paper & Plastic Products)
* **Mustard:** Aisle 6 (Condiments)
* **Potatoes:** Aisle 9 (Produce) 

Please note that these are general aisle locations and may vary depending on the specific grocery store you are visiting. It's always a good idea to check the store directory or ask an employee for assistance if you can't find what you're looking for.


In [31]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Michael's Grocery Store Aisle Layout:

Based on the provided information, here's where you can find the items you listed:

* **Paper plates:** You can find paper plates in **Aisle 17 (Household & Cleaning Supplies)**. 
* **Mustard:** Mustard is typically located in **Aisle 8 (Condiments & Spices)**.
* **Potatoes:** You can find potatoes in two locations:
    * **Aisle 2 (Vegetables)** if you're looking for fresh potatoes.
    * **Aisle 14 (Frozen Foods)** if you're looking for frozen potato products like french fries or tater tots. 



# Task 9. Structure prompts with prefixes or tags
Review the following prompt for a hypothetical text-based dating application. It contains several prompt components, including:

defining a persona
specifying instructions
providing multiple pieces of contextual information for the main user and potential matches.
Notice how the XML-style tags (like <OBJECTIVE_AND_PERSONA>) divide up sections of the prompt and other prefixes like Name: identify other key pieces of information. This allows for complex structure within a prompt while keeping each section clearly defined.

Copy this code block to a new cell and run it to see the output


In [32]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Allison, I think I might have found the perfect match for you! Felix is also a big fan of classical music, especially Beethoven. You two could spend hours discussing your favorite composers and pieces. He also loves going to the beach, just like you enjoy swimming. And guess what? He's a whiz in the kitchen and makes a mean spaetzle, which is a type of German pasta. You both love Italian food, so maybe you can even have a fun night making homemade pasta and enjoying a delicious Italian meal together. 



# Task 10. Use system instructions
You can include prompt components like those you've explored above in each call to a model, or you can pass them to the model upon instantiation as system instructions.

Paste the code below into a new code cell and run it.
Notice how the prompt passed to the generate_content() function doesn't mention music at all, but the model still responds based on the persona & instructions passed to it as a system_instruction.

In [33]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh my,  music history is overflowing with fascinating figures worth studying! Where should we even begin?  

**Composers?**  From the Baroque giants like **Bach** with his intricate fugues, and **Handel**, renowned for his operas, to the enigmatic **Mozart** and the revolutionary **Beethoven**, there's a lifetime of study right there!  

**Let's not forget the innovators:** **Debussy** with his impressionistic soundscapes, **Stravinsky**, who practically redefined orchestral music with "The Rite of Spring", and **Schoenberg**, who turned tonality on its head with his twelve-tone technique!

**But wait, there's more!** Dive into the blues with legends like **Robert Johnson** and **Muddy Waters**, delve into the world of jazz with giants like **Louis Armstrong**, **Billie Holiday**, and **Charlie Parker**. And don't even get me started on the explosion of rock and roll with figures like **Chuck Berry**, **Elvis Presley**, **The Beatles**... the list goes on!

**The beauty of music histor

# Task 11. Demonstrate Chain-of-Thought
Large language models predict what language should follow other language, but they cannot think through cause and effect in the world outside of language. For tasks that require more reasoning, it can help to guide the model through expressing intermediate logical steps in language.

Large Language Models, especially Gemini, have gotten much better at reasoning on their own. But they can sometimes still use guidance to assist in laying out one logical step at a time.

Notice in the code cell below that you will pass a generation_config parameter to the generate_content() function. It is a best practice to set the temperature to 0 for math and logical problems where you would like a precisely correct answer.
Copy and run the following code cell and view its output.

In [34]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

## Today's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

## Tomorrow's Production:

* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory (reconfigured) + 1 factory * 30 units/day/factory (half output) = 90 units/day
* Low efficiency factories: 1 factory * 15 units/day/factory (half output) = 15 units/day
* **Total production tomorrow: 300 units/day + 90 units/day + 15 units/day = 405 units/day** 



# Task 12. Break down complex tasks
Often, complex tasks require multiple steps to work through them, even for us humans! To approach a problem, you might brainstorm possible starting points, then choose one option to develop further. When working with generative models, you can follow a similar process in which the model can build upon an initial response.

In [35]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## Metaphors Comparing TPUs and GPUs:

1. **Assembly Line vs. Swiss Army Knife:** 
    * **TPU:** An assembly line, highly specialized for one task, performing it with incredible speed and efficiency. 
    * **GPU:** A Swiss Army Knife, versatile and capable of handling various tasks, but not as fast or efficient as a dedicated tool.

2. **Superhighway vs. City Streets:**
    * **TPU:** A superhighway, designed for high-speed, unidirectional data flow, ideal for specific AI tasks.
    * **GPU:** A complex network of city streets, handling diverse traffic and calculations, but with potential for congestion.

3. **Laser Scalpel vs. Utility Knife:**
    * **TPU:** A precise laser scalpel, delivering focused power for specific surgical procedures (AI tasks).
    * **GPU:** A multi-purpose utility knife, adaptable for various cutting needs (computations), but less precise for delicate tasks.

4. **Race Car vs. SUV:**
    * **TPU:** A race car, built for speed on a specific track (AI task), 

In [36]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## Metaphor I connect with most:

From the provided options, the metaphor that resonates most with my understanding of TPUs and GPUs is **"Race Car vs. SUV"**. 

This analogy effectively captures the key differences between the two technologies:

* **TPU as a Race Car:** Like a race car built for speed on a specific track, TPUs excel in specific AI tasks. They are highly optimized for these tasks, delivering blistering performance within their domain. However, their specialized nature limits their applicability to other tasks.
* **GPU as an SUV:** GPUs, like SUVs, offer greater versatility. They can handle diverse computational needs, making them suitable for various applications beyond AI. However, this versatility comes at the cost of raw speed. On specialized AI tasks, GPUs may not match the performance of TPUs.

This metaphor aligns with my visual imagination because it invokes a clear picture of the trade-off between specialization and versatility. The race car, while incredibly f

In [37]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## Decoding the Performance Puzzle: Race Car vs. SUV - A Dive into TPUs and GPUs

In the world of Artificial Intelligence, processing power reigns supreme. But navigating the landscape of processing choices can be as daunting as finding the right vehicle for your needs. Do you opt for the raw speed of a race car, or the versatile practicality of an SUV? This same dilemma applies to the choice between Tensor Processing Units (TPUs) and Graphics Processing Units (GPUs). 

Much like the race car on a specific track, TPUs are built for lightning-fast performance for specific AI tasks. They are the single-minded champions of their domain, delivering unparalleled speed within their specialized area. However, like the race car's lack of adaptability, TPUs struggle to navigate beyond their defined tracks. This is where the adaptable SUV-like nature of GPUs comes in. GPUs, much like their vehicular counterparts, offer greater versatility. They can handle diverse computational needs, making them